Importing Required Libraries in python 

In [1]:
import pandas as pd
import numpy as np

In [2]:
lane_dep_warn = pd.read_csv("cas_ldw.csv")
forw_coll_warn = pd.read_csv("cas_fcw.csv")
pedes_coll_warn = pd.read_csv("cas_pcw.csv")
head_motr_warn = pd.read_csv("cas_hmw.csv")

Part 1 : Analyzing each Alert with the Wearther Data

In [3]:
lane_dep_warn.iloc[:,3:].corr(method = "pearson",numeric_only = True)[0:1]

,Count,T,DP,T-DP,H,WS
Count,1.0,0.064756,0.00751,0.047396,-0.048875,0.090514


In [4]:
forw_coll_warn.iloc[:,3:].corr(method = "pearson",numeric_only = True)[0:1]

,Count,T,DP,T-DP,H,WS
Count,1.0,0.066059,0.032729,0.03414,-0.050141,0.061911


In [5]:
pedes_coll_warn.iloc[:,3:].corr(method = "pearson",numeric_only = True)[0:1]

,Count,T,DP,T-DP,H,WS
Count,1.0,-0.145038,-0.0567,-0.085789,0.08339,-0.037


In [6]:
head_motr_warn.iloc[:,3:].corr(method = "pearson",numeric_only = True)[0:1]

,Count,T,DP,T-DP,H,WS
Count,1.0,-0.034771,-0.022,-0.016919,0.011579,0.088159


Here we can see clearly thatt the weather data plays no effect on any of the different alerts

Part - 2 : Analyzing the average speed of each alert

In [7]:
data = pd.read_csv("unnati_phase1_data_revised.csv")
data.head()

,Alert_ID,Alert,Date,Time,Lat,Long,Vehicle,Speed
0,100001,cas_ldw,01/06/2022,06:05:10,12.878990,80.080324,805,13
1,100002,cas_ldw,01/06/2022,06:06:41,12.873922,80.078184,805,59
2,100003,cas_ldw,01/06/2022,06:07:12,12.867110,80.075508,805,58
3,100004,cas_ldw,01/06/2022,06:07:50,12.855921,80.069229,805,58
4,100005,cas_ldw,01/06/2022,06:08:17,12.852359,80.066607,805,58


In [8]:
speed = data[['Alert','Speed']].groupby(['Alert']).mean(["Speed"])
speed.head()

,Speed
Alert,
cas_fcw,36.366102
cas_hmw,35.316596
cas_ldw,50.923496
cas_pcw,17.527328


In [9]:
alert_type =  data[['Vehicle','Date','Alert','Speed']].groupby(['Vehicle','Alert']).agg(count=('Alert','size'), mean_speed = ('Speed','mean'))
print(alert_type)

                 count  mean_speed
Vehicle Alert                     
805     cas_fcw    158   33.879747
        cas_hmw   4227   34.408091
        cas_ldw   2052   49.559454
        cas_pcw    438   17.812785
1995    cas_fcw      1    0.000000
        cas_hmw      8   14.500000
        cas_ldw      8   42.625000
        cas_pcw      1    0.000000
2846    cas_fcw    145   39.613793
        cas_hmw   2409   37.578248
        cas_ldw   2741   51.639183
        cas_pcw    391   18.636829
3143    cas_fcw     82   33.634146
        cas_hmw   1567   34.671347
        cas_ldw    876   51.350457
        cas_pcw    379   18.918206
5339    cas_fcw    204   37.259804
        cas_hmw   4117   35.212048
        cas_ldw    754   51.625995
        cas_pcw    767   16.134289


Here We can see that across vehicles the data for average speed are similar

Part - 3 : Filtering Hospitals close to the accident zones and also on their ability to handle emergencies

In [10]:
hospitals = pd.read_csv("hosp_directory.csv")
print(hospitals)

                X          Y  objectid health_facility_name  \
0       78.564177  12.492635         1     THIRUPATHUR UPHC   
1       78.535451  12.266359         2           UTHANGARAI   
2       78.511082  11.045766         3             JAYANTHI   
3       78.559638  11.193647         4              KUMARAN   
4       78.526689  11.439301         5         Sendarapatti   
...           ...        ...       ...                  ...   
165106  76.465500  23.202200    165107          SHC Chakrod   
165107  78.916286  23.120122    165108            PHC Dobhi   
165108  79.905277  23.202764    165109          SHC Karmeta   
165109  80.002659  23.120099    165110           SHC Umaria   
165110  76.903801  23.120802    165111           PHC Amlaha   

                                address                      street  \
0                    1 THIRUPATHUR UPHC  Chairman Laxshmanan Street   
1       Government Hospital, Uthangarai                  Uthangarai   
2                             

In [11]:
filter = hospitals['state_name'] == "Tamil Nadu"
filter1= hospitals['lat'] >=12.34
filter2= hospitals['lat'] <=13.19
filter3= hospitals['lon'] <=80.32
filter4= hospitals['lon'] >=79.78
filter5= hospitals['facility_type']  == 'Primary Health Centre'
filter6= hospitals['facility_type']  == 'District Hospital'
filter7= hospitals['facility_type']  == 'Sub-District Hospital'
hospitals_tamilnadu = hospitals.where(filter & filter1 & filter2 & filter3 & filter4).dropna()
hospitals_tamilnadu = hospitals_tamilnadu.where(filter5 | filter6 | filter7).dropna()
print(hospitals_tamilnadu)

               X          Y  objectid health_facility_name  \
19231  80.233987  13.070108   19232.0          KILPAUK UHP   
19233  80.275224  13.029925   19234.0         SANTHOME UHP   
19234  80.259610  13.017820   19235.0      NEHRU NAGAR UHP   
19235  80.245294  13.075499   19236.0     MANGALAPURAM UHP   
19236  80.246436  13.002852   19237.0              VHS UHP   
...          ...        ...       ...                  ...   
47451  80.222121  13.055125   47452.0       VADAPALANI UHP   
47452  80.200147  13.100923   47453.0  VILLIWAKKAM  II UHP   
47456  80.214888  13.028199   47457.0         SAIDAPET UHP   
47459  80.204801  13.029936   47460.0    SAIDAPET WEST UHP   
47633  79.780134  13.131521   47634.0       Thiruvelangadu   

                            address                               street  \
19231                             1                        Temple Street   
19233                           14A   APPU 2nd LANE,Mylapore,Chennai-04.   
19234                      

We have now got a list of 106 hospitals to work with in QGIS instead of 1,65,111